## Seção 0: Configuração e Funções Auxiliares

Nesta seção, importamos as bibliotecas necessárias, definimos as variáveis de configuração (como caminhos de arquivos) e criamos funções de apoio que serão usadas ao longo do notebook.

In [1]:
configuracoesConexao = {
    'Base': 'FactaDB',
    'User': 'Facta',
    'Pass': 'F@cta019',
    'Server': 'factasqlserver.database.windows.net',
}

In [2]:
ConexaoDatabase = None

import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import urllib.parse

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={configuracoesConexao['Server']};"
    f"DATABASE={configuracoesConexao['Base']};"
    f"UID={configuracoesConexao['User']};"
    f"PWD={configuracoesConexao['Pass']};"
    f"TrustServerCertificate=yes;"
)

conn = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)

In [3]:
ExibicaoResultadosDataFrame = None

from IPython.display import display, HTML

def show_df(df, title=''):
    styles = """<style>.dataframe-container { overflow-x: auto; max-height: 400px; overflow-y: auto; } table { border-collapse: collapse; width: 100%; font-size: 11px; } th, td { border: 1px solid #ddd; padding: 4px; white-space: nowrap; } th { background-color: gray; position: sticky; top: 0; z-index: 2; } </style>"""
    html = styles
    if title:
        html += f'<h3>{title}</h3>'
    html += f'<p>Shape: {df.shape}</p>'
    html += '<div class="dataframe-container">' + df.to_html(index=True) + '</div>'
    display(HTML(html))

In [ ]:
ConsultaPessoaEnderecoNaoPossuemDS_Cidade = None

numeroLote = 12307
IdRegistradoraLoteDetalhe = 36333038

query = f"""
;WITH DetalheErro AS (
    SELECT TOP 1
        RLD.ID_Endosso
    FROM RegistradoraLoteDetalhe RLD WITH (NOLOCK)
    JOIN RegistradoraLoteDetalheErro RDE WITH (NOLOCK) ON RDE.ID_RegistradoraLoteDetalhe = RLD.ID_RegistradoraLoteDetalhe
    WHERE RLD.ID_RegistradoraLoteDetalhe = {IdRegistradoraLoteDetalhe}
),
C1 AS (
	SELECT S.ID_Endosso
	FROM SaneamentoSROApolice S
    JOIN DetalheErro DE ON DE.ID_Endosso = S.ID_Endosso
	WHERE S.NR_AuxiliarControleLote = {numeroLote}
),
PE_A_Atualizar AS (
	SELECT
		   PE.ID_PessoaEndereco,
	       PE.DS_Endereco,
	       PE.DS_Numero,
	       PE.DS_Bairro,
	       PE.NR_CEP,
	       PE.DS_UF
	FROM Segurado S1 WITH (NOLOCK)
	JOIN PessoaEndereco PE WITH (NOLOCK) ON S1.ID_PessoaEndereco = PE.ID_PessoaEndereco
	JOIN Endosso E WITH (NOLOCK) ON E.ID_Endosso = S1.ID_Endosso
	JOIN C1 WITH (NOLOCK) ON C1.ID_Endosso = E.ID_Endosso
)
SELECT PE.*
FROM PessoaEndereco PE WITH (NOLOCK)
JOIN PE_A_Atualizar PEAU WITH (NOLOCK) ON PEAU.ID_PessoaEndereco = PE.ID_PessoaEndereco
ORDER BY PE.ID_PessoaEndereco, PE.NR_CEP, PE.DS_Endereco
"""
dfLote = pd.read_sql(query, conn)
dfLote.count()
dfLote

In [ ]:
ConsultaPessoaEnderecoNaoPossuemDS_Cidade = None

numeroLote = 12336

query = f"""
;WITH C1 AS (
	SELECT ID_Endosso
	FROM SaneamentoSROApolice S
	WHERE S.NR_AuxiliarControleLote = {numeroLote}
	    AND FL_Processado = 0 and NR_AuxiliarControleLote BETWEEN 12168 and 12170
),
PE_A_Atualizar AS (
	SELECT
		   PE.ID_PessoaEndereco,
	       PE.DS_Endereco,
	       PE.DS_Numero,
	       PE.DS_Bairro,
	       PE.NR_CEP,
	       PE.DS_UF
	FROM Segurado S1 WITH (NOLOCK)
	JOIN PessoaEndereco PE WITH (NOLOCK) ON S1.ID_PessoaEndereco = PE.ID_PessoaEndereco
	JOIN Endosso E WITH (NOLOCK) ON E.ID_Endosso = S1.ID_Endosso
	JOIN C1 WITH (NOLOCK) ON C1.ID_Endosso = E.ID_Endosso
	WHERE COALESCE(PE.DS_Cidade, '') = ''
	  AND COALESCE(PE.DS_Endereco, '') != ''
),
CEPs_Repetidos AS (
    SELECT
        NR_CEP,
        COUNT(DISTINCT DS_Endereco) AS Quantidade_Enderecos
    FROM PE_A_Atualizar WITH (NOLOCK)
    GROUP BY NR_CEP
    HAVING COUNT(DISTINCT DS_Endereco) > 1
)
SELECT PE.*
FROM PessoaEndereco PE WITH (NOLOCK)
JOIN PE_A_Atualizar PEAU WITH (NOLOCK) ON PEAU.ID_PessoaEndereco = PE.ID_PessoaEndereco
ORDER BY PE.ID_PessoaEndereco, PE.NR_CEP, PE.DS_Endereco
"""
dfLote = pd.read_sql(query, conn)
dfLote.count()

In [ ]:
numeroLoteInicial = 12168
numeroLoteFinal = 12350
filtro = '.enderecoPessoaApolice.'

query = f"""
;WITH DetalheErro AS (
    SELECT
        RL.ID_RegistradoraLote,
        RL.ID_RegistradoraLoteDetalhe,
        RLD.ID_Endosso
    FROM RegistradoraLoteDetalhe RL WITH (NOLOCK)
    CROSS APPLY (
        SELECT TOP 1 T.*
        FROM RegistradoraLoteDetalheErro T WITH (NOLOCK)
        WHERE T.ID_RegistradoraLoteDetalhe = RL.ID_RegistradoraLoteDetalhe
          AND T.DT_Erro >= dbo.getDateLocal() - 1
          AND T.DS_Erro LIKE '%{filtro}%'
        ORDER BY T.ID_RegistradoraLoteDetalheErro DESC
    ) AS TE
    JOIN RegistradoraLoteDetalhe RLD WITH (NOLOCK)
         ON RLD.ID_RegistradoraLoteDetalhe = RL.ID_RegistradoraLoteDetalhe
    WHERE RL.ID_RegistradoraLote BETWEEN {numeroLoteInicial} AND {numeroLoteFinal}
      AND COALESCE(RLD.FL_Sucesso, 0) = 0
),
PE_A_Atualizar AS (
    SELECT
        PE.ID_PessoaEndereco,
        PE.DS_Endereco,
        PE.DS_Numero,
        PE.DS_Bairro,
        PE.NR_CEP,
        PE.DS_UF,
        DE.ID_RegistradoraLote,
        DE.ID_RegistradoraLoteDetalhe,
        F.ID_Prime
    FROM Segurado S WITH (NOLOCK)
    JOIN PessoaEndereco PE WITH (NOLOCK)
         ON S.ID_PessoaEndereco = PE.ID_PessoaEndereco
    JOIN Endosso E WITH (NOLOCK)
         ON E.ID_Endosso = S.ID_Endosso
    JOIN DetalheErro DE ON DE.ID_Endosso = E.ID_Endosso
    JOIN Fatura F WITH (NOLOCK)
         ON F.ID_Fatura = E.ID_Fatura
    JOIN ApoliceFatura AF WITH (NOLOCK)
         ON AF.ID_ApoliceFatura = F.ID_ApoliceFatura
)
SELECT
    PE.*,
    PEAU.ID_Prime,
    PEAU.ID_RegistradoraLote,
    PEAU.ID_RegistradoraLoteDetalhe
FROM PessoaEndereco PE WITH (NOLOCK)
JOIN PE_A_Atualizar PEAU WITH (NOLOCK)
     ON PEAU.ID_PessoaEndereco = PE.ID_PessoaEndereco
ORDER BY PE.ID_PessoaEndereco, PE.NR_CEP, PE.DS_Endereco;
"""
dfLote = pd.read_sql(query, conn)
dfLote = dfLote.fillna('')
show_df(dfLote)

,ID_PessoaEndereco,ID_Pessoa,NR_CEP,DS_Endereco,DS_Numero,DS_Complemento,DS_Bairro,DS_Cidade,DS_UF,ID_Prime,ID_RegistradoraLote,ID_RegistradoraLoteDetalhe
0,2616958,39,88900000,ESTRADA GERAL,1,0,CENTRO,,SC,343127,12170,29586999
1,2619154,699,93300000,RIO SAO FRANCISCO,25,0,CANUDOS,,RS,346725,12170,29590158
2,2619154,699,93300000,RIO SAO FRANCISCO,25,0,CANUDOS,,RS,348295,12170,29591517
3,2629188,4299,93260000,RUA SAO SEBASTIAO DO CAI,105,0,ESTEIO,,RS,349040,12169,29563290
4,2629188,4299,93260000,RUA SAO SEBASTIAO DO CAI,105,0,ESTEIO,,RS,349040,12170,29592153
5,2652077,13134,97509264,QUADRA A,123,0,CENTRO,,RS,340871,12170,29585011
6,2657453,14928,95780000,R PAPOULAS,275,CASA,BELA VISTA,,RS,340169,12170,29584355
7,2662902,17223,93800000,PLINIO BRASIL MILANO,100,0,SETE DE SETEMBRO,,RS,351021,12169,29564342
8,2662902,17223,93800000,PLINIO BRASIL MILANO,100,0,SETE DE SETEMBRO,,RS,351021,12170,29593814
9,2663684,17566,95600000,ESTRADA DAS FLORES,850,0,MORRO DA PEDRA,,RS,340083,12170,29584278


In [10]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np

# --- Variáveis de Configuração ---
# ARQUIVO_ORIGEM_CLIENTES = r"C:\Dev\Testes\12269_PessoaEnderecoValidacao.csv"
ARQUIVO_MUNICIPIOS_IBGE = r"C:\Dev\Testes\CNEFE\RELATORIO_DTB_BRASIL_2024_MUNICIPIOS.csv"
PASTA_CNEFE_BASE_DIR = r"C:\Dev\Testes\CNEFE_ZIP"

# --- Configurações do Pandas ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Seção 1: Carregamento e Preparação dos Dados

Aqui, carregamos os dados de três fontes distintas:
1.  **CNEFE (IBGE):** Nossa base de referência de endereços, extraída de múltiplos arquivos ZIP.
2.  **Municípios (IBGE):** Um arquivo de de-para entre códigos e nomes de municípios.
3.  **Endereços de Origem:** O arquivo CSV com os endereços que queremos validar.

As funções abaixo encapsulam o processo de carregamento e a limpeza inicial.

In [11]:
import os
import zipfile
from io import TextIOWrapper

def carregar_dados_cnefe_otimizado(lista_pastas_base, ceps_filtro):
    """
    Carrega dados do CNEFE de forma otimizada, filtrando por CEP e colunas
    durante a leitura para economizar memória.
    """
    # Carregamos apenas as colunas estritamente necessárias para a validação
    colunas_necessarias = [
        'COD_MUNICIPIO', 'COD_UF', 'CEP', 'DSC_LOCALIDADE',
        'NOM_TIPO_SEGLOGR', 'NOM_TITULO_SEGLOGR', 'NOM_SEGLOGR', 'NUM_ENDERECO'
    ]

    dataframes = []
    ceps_set = set(ceps_filtro) # Usar um set é muito mais rápido para buscas (O(1))

    for pasta_base in lista_pastas_base:
        print(f"--- Iniciando varredura em: {pasta_base} ---")
        if not os.path.isdir(pasta_base):
            print(f"Aviso: O diretório '{pasta_base}' não foi encontrado.")
            continue

        for root, _, files in os.walk(pasta_base):
            for file in files:
                if file.endswith(".zip"):
                    caminho_zip = os.path.join(root, file)
                    try:
                        with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
                            for nome_csv in zip_ref.namelist():
                                if nome_csv.lower().endswith(".csv"):
                                    with zip_ref.open(nome_csv) as arquivo_csv:
                                        # Otimização: usecols + filtro na leitura
                                        df_chunk = pd.read_csv(
                                            TextIOWrapper(arquivo_csv, encoding='utf-8'),
                                            sep=';', dtype=str,
                                            usecols=colunas_necessarias,
                                            low_memory=False
                                        ).fillna("")

                                        # Filtra o chunk pelos CEPs de interesse
                                        df_filtrado = df_chunk[df_chunk['CEP'].isin(ceps_set)]

                                        if not df_filtrado.empty:
                                            dataframes.append(df_filtrado)
                    except Exception as e:
                        print(f"Erro ao processar {caminho_zip}: {e}")

    if not dataframes:
        # Não é mais um erro fatal, pode ser que nenhum CEP dos clientes foi encontrado nos arquivos
        print("Aviso: Nenhum endereço correspondente aos CEPs de origem foi encontrado nos arquivos CNEFE.")
        return pd.DataFrame(columns=colunas_necessarias)

    df_cnefe_completo = pd.concat(dataframes, ignore_index=True)
    print(f"\n✔ Processamento concluído. Total de {len(df_cnefe_completo)} registros relevantes carregados.")
    return df_cnefe_completo

def preparar_base_referencia(df_cnefe, df_municipios, df_ufs):
    """
    Enriquece e padroniza a base do CNEFE (agora muito menor) para a validação.
    """
    if df_cnefe.empty:
        print("Base CNEFE vazia, pulando preparação.")
        # Retorna um DataFrame vazio com a estrutura esperada
        return pd.DataFrame(columns=['NR_CEP', 'REF_Logradouro', 'REF_Numero', 'REF_Bairro', 'REF_Cidade', 'REF_UF'])

    df_ref = df_cnefe.rename(columns={
        'CEP': 'NR_CEP', 'NOM_SEGLOGR': 'DS_Logradouro_Nome',
        'NUM_ENDERECO': 'DS_Numero', 'DSC_LOCALIDADE': 'DS_Bairro'
    })
    df_municipios = df_municipios.rename(columns={'Codigo_Municipio': 'COD_MUNICIPIO', 'Nome_Municipio': 'DS_Cidade'})

    # Os merges agora são muito mais rápidos
    df_ref = df_ref.merge(df_municipios[['COD_MUNICIPIO', 'DS_Cidade']], on='COD_MUNICIPIO', how='left')
    df_ref = df_ref.merge(df_ufs, on='COD_UF', how='left')

    # Esta operação agora é segura e rápida, pois o df_ref é pequeno
    for col in ['DS_Logradouro_Nome', 'DS_Bairro', 'DS_Cidade', 'DS_UF', 'NOM_TIPO_SEGLOGR', 'NOM_TITULO_SEGLOGR']:
         if col in df_ref.columns:
            df_ref[col] = df_ref[col].str.strip().str.upper()

    df_ref['DS_Logradouro_Completo'] = (
        df_ref['NOM_TIPO_SEGLOGR'].fillna('') + ' ' +
        df_ref['NOM_TITULO_SEGLOGR'].fillna('') + ' ' +
        df_ref['DS_Logradouro_Nome'].fillna('')
    ).str.replace(r'\s+', ' ', regex=True).str.strip()

    df_ref['DS_Numero'] = df_ref['DS_Numero'].replace({'0': 'SN', '': 'SN'})

    df_ref = df_ref[[
        'NR_CEP', 'DS_Logradouro_Completo', 'DS_Numero',
        'DS_Bairro', 'DS_Cidade', 'DS_UF'
    ]].rename(columns={
        'DS_Logradouro_Completo': 'REF_Logradouro', 'DS_Numero': 'REF_Numero',
        'DS_Bairro': 'REF_Bairro', 'DS_Cidade': 'REF_Cidade', 'DS_UF': 'REF_UF'
    })

    df_ref = df_ref.drop_duplicates()
    print("✔ Base de referência preparada.")
    return df_ref

In [12]:
# --- PASSO 1: Carregar os dados de origem (pequenos) e extrair os CEPs ---
print("--- Carregando dados de origem e de apoio ---")
#df_origem = pd.read_csv(ARQUIVO_ORIGEM_CLIENTES, sep=',', dtype=str).fillna('')
df_origem = dfLote

for col in df_origem.columns:
    if df_origem[col].dtype == 'object':
        df_origem[col] = df_origem[col].str.strip().str.upper()

print("Colunas encontradas no arquivo de origem:", df_origem.columns.tolist())

ufs_no_arquivo = df_origem['DS_UF'].dropna().unique().tolist()
print(f"\nUFs encontrados no arquivo de origem para processamento: {ufs_no_arquivo}")

pastas_a_processar = [os.path.join(PASTA_CNEFE_BASE_DIR, uf) for uf in ufs_no_arquivo if uf]
print(f"Pastas CNEFE que serão carregadas: {pastas_a_processar}")

ceps_de_interesse = df_origem['NR_CEP'].unique().tolist()
print(f"\nEncontrados {len(ceps_de_interesse)} CEPs únicos para otimizar a busca.")

# --- PASSO 2: Carregar dados de apoio (municípios e UFs) ---
df_municipios_raw = pd.read_csv(ARQUIVO_MUNICIPIOS_IBGE, sep=';', dtype=str).fillna('')
ufs_codigos = [
    ('AC', '12'),
    ('AL', '27'),
    ('AP', '16'),
    ('AM', '13'),
    ('BA', '29'),
    ('CE', '23'),
    ('DF', '53'),
    ('ES', '32'),
    ('GO', '52'),
    ('MA', '21'),
    ('MT', '51'),
    ('MS', '50'),
    ('MG', '31'),
    ('PA', '15'),
    ('PB', '25'),
    ('PR', '41'),
    ('PE', '26'),
    ('PI', '22'),
    ('RJ', '33'),
    ('RN', '24'),
    ('RS', '43'),
    ('RO', '11'),
    ('RR', '14'),
    ('SC', '42'),
    ('SP', '35'),
    ('SE', '28'),
    ('TO', '17')
]

df_ufs_map = pd.DataFrame(ufs_codigos, columns=['DS_UF', 'COD_UF'])

# --- PASSO 3: Chamar o carregador otimizado do CNEFE com o filtro de CEPs ---
df_cnefe_raw = carregar_dados_cnefe_otimizado(pastas_a_processar, ceps_de_interesse)

# --- PASSO 4: Preparar a base de referência (agora muito menor) ---
df_referencia = preparar_base_referencia(df_cnefe_raw, df_municipios_raw, df_ufs_map)
#show_df(df_referencia.head(), title="Amostra da Base de Referência (Filtrada e Final)")
#show_df(df_origem, title="Dados de Origem a Validar")

--- Carregando dados de origem e de apoio ---
Colunas encontradas no arquivo de origem: ['ID_PessoaEndereco', 'ID_Pessoa', 'NR_CEP', 'DS_Endereco', 'DS_Numero', 'DS_Complemento', 'DS_Bairro', 'DS_Cidade', 'DS_UF', 'ID_Prime', 'ID_RegistradoraLote', 'ID_RegistradoraLoteDetalhe']

UFs encontrados no arquivo de origem para processamento: ['SC', 'RS', 'PB', 'BA', 'GO', 'MA', 'CE', 'SP', 'RJ', 'ES', 'MG', 'AL', 'RR', 'PR', 'MS', 'PA', 'AC', 'RN', 'AM', 'PE', 'MT', 'RO', 'DF', 'PI', 'TO', 'SE']
Pastas CNEFE que serão carregadas: ['C:\\Dev\\Testes\\CNEFE_ZIP\\SC', 'C:\\Dev\\Testes\\CNEFE_ZIP\\RS', 'C:\\Dev\\Testes\\CNEFE_ZIP\\PB', 'C:\\Dev\\Testes\\CNEFE_ZIP\\BA', 'C:\\Dev\\Testes\\CNEFE_ZIP\\GO', 'C:\\Dev\\Testes\\CNEFE_ZIP\\MA', 'C:\\Dev\\Testes\\CNEFE_ZIP\\CE', 'C:\\Dev\\Testes\\CNEFE_ZIP\\SP', 'C:\\Dev\\Testes\\CNEFE_ZIP\\RJ', 'C:\\Dev\\Testes\\CNEFE_ZIP\\ES', 'C:\\Dev\\Testes\\CNEFE_ZIP\\MG', 'C:\\Dev\\Testes\\CNEFE_ZIP\\AL', 'C:\\Dev\\Testes\\CNEFE_ZIP\\RR', 'C:\\Dev\\Testes\\CNEFE_ZIP\

## Seção 2: Motor de Validação de Endereços

Esta é a seção principal. A função `validar_enderecos` orquestra todo o processo de validação de forma vetorizada, implementando a lógica da "árvore de decisões".

**Estratégia:**
1.  **Merge por CEP:** Cruzamos a base de origem com a de referência usando o CEP como chave. Endereços com CEP inválido não encontrarão correspondência.
2.  **Validação de Campos:** Com os dados lado a lado, comparamos logradouro, bairro e cidade.
3.  **Agregação:** Como um CEP pode ter múltiplos resultados, agrupamos pelo ID do endereço de origem e verificamos se *alguma* das correspondências é válida.
4.  **Geração de Status:** Com base nos resultados da validação, atribuímos um status final e uma sugestão clara para cada endereço.

In [13]:
import numpy as np
import pandas as pd
from rapidfuzz import fuzz
from unidecode import unidecode

def normalize_street_name(series):
    """Aplica uma série de limpezas em uma coluna de logradouros para melhorar a comparação."""
    street_types_to_remove = r'\b(RUA|AVENIDA|AV|R|ALAMEDA|TRAVESSA|TRV|PRACA|PC|PCA|LARGO|ESTRADA|RODOVIA)\b'
    return series.astype(str).str.upper().apply(unidecode).str.replace(street_types_to_remove, '', regex=True).str.replace(r'[^A-Z0-9\s]', '', regex=True).str.strip()

def find_best_match(group):
    """
    Função personalizada para ser aplicada a cada grupo de CEP.
    Calcula a similaridade e retorna a melhor correspondência.
    """
    if group['REF_Logradouro_Norm'].isnull().all():
        return pd.Series({
            'Score_Logradouro': 0,
            'REF_Logradouro': None,
            'REF_Bairro': None,
            'REF_Cidade': None
        })

    origin_street_norm = group['DS_Endereco_Norm'].iloc[0]
    scores = group['REF_Logradouro_Norm'].apply(lambda ref_street: fuzz.token_sort_ratio(origin_street_norm, ref_street))
    best_match_index = scores.idxmax()

    return pd.Series({
        'Score_Logradouro': scores[best_match_index],
        'REF_Logradouro': group.loc[best_match_index, 'REF_Logradouro'],
        'REF_Bairro': group.loc[best_match_index, 'REF_Bairro'],
        'REF_Cidade': group.loc[best_match_index, 'REF_Cidade']
    })

def validar_enderecos(df_origem, df_ref):
    """
    Motor de validação otimizado que usa groupby().apply() para performance e gera
    sugestões e status compostos para máxima precisão.
    """
    print("Iniciando validação inteligente com correspondência parcial...")

    # --- PASSO 1: Preparação e Normalização ---
    df_origem_com_id = df_origem.reset_index().rename(columns={'index': 'ID_Linha_Original'})

    print("Passo 1: Normalizando nomes de logradouros para comparação.")
    df_origem_com_id['DS_Endereco_Norm'] = normalize_street_name(df_origem_com_id['DS_Endereco'])

    df_ref = df_ref.copy()
    if 'REF_Logradouro' in df_ref.columns:
        df_ref['REF_Logradouro_Norm'] = normalize_street_name(df_ref['REF_Logradouro'])
    else:
        df_ref['REF_Logradouro_Norm'] = ''

    # --- PASSO 2: Otimização de Performance com GroupBy e Apply Personalizado ---
    print("Passo 2: Cruzando dados e identificando a melhor correspondência por grupo.")
    df_merged = pd.merge(df_origem_com_id, df_ref, on='NR_CEP', how='left')

    if df_merged.empty:
        df_results = pd.DataFrame(columns=['ID_Linha_Original', 'Score_Logradouro', 'REF_Logradouro', 'REF_Bairro', 'REF_Cidade'])
    else:
        # CORREÇÃO: Adicionado 'include_groups=False' para alinhar com o futuro padrão do Pandas e remover o warning.
        df_results = df_merged.groupby('ID_Linha_Original').apply(find_best_match, include_groups=False).reset_index()

    # Junta os resultados da busca de volta ao DataFrame de origem
    df_final = pd.merge(df_origem_com_id, df_results, on='ID_Linha_Original')
    df_final['Score_Logradouro'] = df_final['Score_Logradouro'].round(2)

    # --- PASSO 3: Geração de Flags, Status e Sugestões Precisas ---
    print("Passo 3: Construindo Status e Sugestões compostas.")

    SIMILARITY_THRESHOLD = 85

    df_final['CEP_Valido'] = df_final['REF_Cidade'].notna()
    df_final['Logradouro_Valido'] = (df_final['Score_Logradouro'] >= SIMILARITY_THRESHOLD)
    df_final['Bairro_Valido'] = (df_final['DS_Bairro'] == df_final['REF_Bairro'])
    df_final['Cidade_Valida'] = (df_final['DS_Cidade'] == df_final['REF_Cidade'])
    df_final['REF_Cidade'] = df_final['REF_Cidade'].fillna('')

    erros_log = ~df_final['Logradouro_Valido'] & df_final['CEP_Valido']
    erros_bairro = ~df_final['Bairro_Valido'] & df_final['CEP_Valido']
    erros_cidade = ~df_final['Cidade_Valida'] & df_final['CEP_Valido']

    status_list = np.where(erros_log, 'Logradouro', '')
    status_list = np.char.add(status_list, np.where(erros_bairro, ', Bairro', ''))
    status_list = np.char.add(status_list, np.where(erros_cidade, ', Cidade', ''))
    status_list = [s.strip(', ') for s in status_list]

    sug_log = np.where(erros_log, "Logradouro Sugerido: '" + df_final['REF_Logradouro'].fillna(''), '')
    sug_bairro = np.where(erros_bairro, "Bairro Correto: '" + df_final['REF_Bairro'].fillna('') + "'", '')
    sug_cidade = np.where(erros_cidade, "Cidade Correta: '" + df_final['REF_Cidade'].fillna('') + "'", '')

    suggestions_list = [', '.join(filter(None, parts)) for parts in zip(sug_log, sug_bairro, sug_cidade)]

    df_final['Status'] = np.where(
        df_final['CEP_Valido'],
        np.where(
            (df_final['Logradouro_Valido'] & df_final['Bairro_Valido'] & df_final['Cidade_Valida']),
            'Endereço Válido',
            'Divergência (' + pd.Series(status_list) + ')'
        ),
        'CEP Inválido/Não Encontrado'
    )
    df_final['Sugestao'] = np.where(df_final['Status'] == 'Endereço Válido', '', pd.Series(suggestions_list))
    df_final.loc[df_final['Status'] == 'CEP Inválido/Não Encontrado', 'Sugestao'] = 'Verificar CEP. Não encontrado na base de referência.'

    # --- PASSO 4: Consolidação Final ---
    print("Passo 4: Consolidando o resultado final.")
    colunas_originais = list(df_origem.columns)
    colunas_validacao = ['Status', 'Sugestao', 'CEP_Valido', 'Logradouro_Valido', 'Bairro_Valido', 'Cidade_Valida', 'Score_Logradouro', 'REF_Cidade']
    df_final = df_final[colunas_originais + colunas_validacao]

    print("✔ Validação inteligente concluída com sucesso.")
    return df_final

## Seção 3: Execução e Análise dos Resultados

Agora, executamos o motor de validação e analisamos o DataFrame resultante, que contém os endereços originais enriquecidos com o status da validação e sugestões de correção.

In [14]:
df_resultado_final = validar_enderecos(df_origem, df_referencia)
df_resultado_final['ID_PessoaEndereco'] = pd.to_numeric(df_resultado_final['ID_PessoaEndereco'], errors='coerce')
df_resultado_final = df_resultado_final.sort_values(by='ID_PessoaEndereco')

Iniciando validação inteligente com correspondência parcial...
Passo 1: Normalizando nomes de logradouros para comparação.
Passo 2: Cruzando dados e identificando a melhor correspondência por grupo.
Passo 3: Construindo Status e Sugestões compostas.
Passo 4: Consolidando o resultado final.
✔ Validação inteligente concluída com sucesso.


In [15]:
show_df(df_resultado_final, title="Resultado Final da Validação de Endereços (Ordenado por ID_PessoaEndereco)")

,ID_PessoaEndereco,ID_Pessoa,NR_CEP,DS_Endereco,DS_Numero,DS_Complemento,DS_Bairro,DS_Cidade,DS_UF,ID_Prime,ID_RegistradoraLote,ID_RegistradoraLoteDetalhe,Status,Sugestao,CEP_Valido,Logradouro_Valido,Bairro_Valido,Cidade_Valida,Score_Logradouro,REF_Cidade
0,2616958,39,88900000,ESTRADA GERAL,1,0,CENTRO,,SC,343127,12170,29586999,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
1,2619154,699,93300000,RIO SAO FRANCISCO,25,0,CANUDOS,,RS,346725,12170,29590158,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
2,2619154,699,93300000,RIO SAO FRANCISCO,25,0,CANUDOS,,RS,348295,12170,29591517,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
3,2629188,4299,93260000,RUA SAO SEBASTIAO DO CAI,105,0,ESTEIO,,RS,349040,12169,29563290,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
4,2629188,4299,93260000,RUA SAO SEBASTIAO DO CAI,105,0,ESTEIO,,RS,349040,12170,29592153,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
5,2652077,13134,97509264,QUADRA A,123,0,CENTRO,,RS,340871,12170,29585011,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
6,2657453,14928,95780000,R PAPOULAS,275,CASA,BELA VISTA,,RS,340169,12170,29584355,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
7,2662902,17223,93800000,PLINIO BRASIL MILANO,100,0,SETE DE SETEMBRO,,RS,351021,12169,29564342,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
8,2662902,17223,93800000,PLINIO BRASIL MILANO,100,0,SETE DE SETEMBRO,,RS,351021,12170,29593814,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,
9,2663684,17566,95600000,ESTRADA DAS FLORES,850,0,MORRO DA PEDRA,,RS,340083,12170,29584278,CEP Inválido/Não Encontrado,Verificar CEP. Não encontrado na base de referência.,False,False,False,False,0.0,


In [16]:
df_resultado_final['DS_Cidade'] = np.where(
    ~df_resultado_final['Cidade_Valida'] & df_resultado_final['REF_Cidade'].notna(),
    df_resultado_final['REF_Cidade'],
    df_resultado_final['DS_Cidade']
)

df_resultado_final['DS_Cidade'] = df_resultado_final['DS_Cidade'].astype("string").str.strip()

df_resultado_final = df_resultado_final[
    df_resultado_final['DS_Cidade'].notna() & (df_resultado_final['DS_Cidade'] != '')
].reset_index(drop=True)

show_df(df_resultado_final)

,ID_PessoaEndereco,ID_Pessoa,NR_CEP,DS_Endereco,DS_Numero,DS_Complemento,DS_Bairro,DS_Cidade,DS_UF,ID_Prime,ID_RegistradoraLote,ID_RegistradoraLoteDetalhe,Status,Sugestao,CEP_Valido,Logradouro_Valido,Bairro_Valido,Cidade_Valida,Score_Logradouro,REF_Cidade
0,5809475,3055738,89980000,,496,,A,CAMPO ERÊ,SC,4664851,12267,34324206,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA DIONISIO CERQUEIRA, Bairro Correto: 'CAMPO ERE', Cidade Correta: 'CAMPO ERÊ'",True,False,False,False,0.0,CAMPO ERÊ
1,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,4731578,12268,34390765,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
2,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,5006676,12273,34664960,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
3,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,5451676,12282,35103268,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
4,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,5844568,12290,35493286,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
5,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,6272746,12299,35919814,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
6,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,6688321,12307,36333038,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
7,5861881,3099888,37580000,,112,,CENTRO,MONTE SIÃO,MG,8177438,12334,37709573,"Divergência (Logradouro, Bairro, Cidade)","Logradouro Sugerido: 'RUA HENRIQUE GENGHINI, Bairro Correto: 'SAO PELEGRINO', Cidade Correta: 'MONTE SIÃO'",True,False,False,False,0.0,MONTE SIÃO
8,6794936,3789874,61814584,,257,,VILA DAS FLORES,PACATUBA,CE,6275027,12299,35922095,Divergência (Logradouro),Logradouro Sugerido: 'RUA 07,True,False,True,True,0.0,PACATUBA


conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={configuracoesConexao['Server']};"
    f"DATABASE={configuracoesConexao['Base']};"
    f"UID={configuracoesConexao['User']};"
    f"PWD={configuracoesConexao['Pass']};"
    f"TrustServerCertificate=yes;"
)
cursor = conn.cursor()
cursor.fast_executemany = True

# Monta a lista de valores (nova cidade, id do endereço)
valores = [
    (row.DS_Cidade, row.ID_PessoaEndereco)
    for _, row in df_resultado_final.iterrows()
        if pd.notna(row.DS_Cidade) and not pd.isna(row.ID_PessoaEndereco) and row.DS_Cidade != ''
]

# Executa update em batch
cursor.executemany(
    "UPDATE PessoaEndereco SET DS_Cidade = ? WHERE ID_PessoaEndereco = ?",
    valores
)

conn.commit()
print(f"Atualizados {len(valores)} registros na tabela PessoaEndereco.")

cursor.close()

In [25]:
import pandas as pd
from fpdf import FPDF
from fpdf.fonts import FontFace
from fpdf.enums import TableBordersLayout, XPos, YPos

def salvar_df_como_pdf(df, caminho_arquivo, titulo=''):
    """
    Salva um DataFrame como um relatório PDF A4 paginado e profissional usando fpdf2,
    com correção para o número de colunas e sintaxe atualizada.

    Args:
        df (pd.DataFrame): O DataFrame a ser salvo.
        caminho_arquivo (str): O caminho completo para o arquivo PDF de saída.
        titulo (str): Um título opcional para o relatório.
    """
    print(f"Iniciando a geração do PDF profissional em: {caminho_arquivo}...")

    if df.empty:
        print("⚠️ DataFrame vazio, nenhum PDF será gerado.")
        return

    try:
        # --- ETAPA 1: Configuração do PDF ---
        pdf = FPDF(orientation="L", unit="mm", format="A3")
        pdf.set_auto_page_break(auto=True, margin=0.1)
        pdf.add_page()
        pdf.set_font("helvetica", "B", 9)

        # --- ETAPA 2: Adiciona o Título ---
        if titulo:
            pdf.cell(0, 10, titulo, new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
            pdf.ln(5)

        # --- ETAPA 3: Renderiza a Tabela ---
        headings_style = FontFace(emphasis="BOLD", color=255, fill_color=(46, 117, 182), size_pt=7)

        # --- CORREÇÃO: Lógica de Largura de Coluna Flexível ---
        desired_widths = {
            'ID_PessoaEndereco': 14,
            'ID_Pessoa': 12,
            'NR_CEP': 15,
            'DS_Endereco': 40,
            'DS_Numero': 10,
            'DS_Complemento': 15,
            'DS_Bairro': 22,
            'DS_Cidade': 10,
            'DS_UF': 8,
            'Status': 50,
            'Sugestao': 90,
            'CEP_Valido': 10,
            'Logradouro_Valido': 10,
            'Bairro_Valido': 10,
            'Cidade_Valida': 10,
            'Score_Logradouro': 10
        }
        # Colunas não especificadas recebem uma largura padrão
        default_width = 15

        # Cria a lista final de larguras na ordem correta das colunas do DataFrame
        col_widths = [desired_widths.get(col, default_width) for col in df.columns]

        # Normaliza as larguras para preencher o total da página
        total_defined_width = sum(col_widths)
        page_width = 399.9
        scaling_factor = page_width / total_defined_width
        col_widths = [w * scaling_factor for w in col_widths]

        with pdf.table(
            borders_layout=TableBordersLayout.HORIZONTAL_LINES,
            col_widths=col_widths, # Usa a lista de larguras agora corrigida e flexível
            headings_style=headings_style,
            line_height=5,
            text_align="LEFT", # Alinhamento padrão para todas as colunas
            width=page_width
        ) as tabela:
            headings = []
            for h in df.columns:
                if h.endswith('_Valido') or h.endswith('_Valida'):
                    headings.append(h.rsplit('_', 1)[0])
                else:
                    headings.append(h.title())
            data_rows = df.values.tolist()

            tabela.row(headings)
            for data_row in data_rows:
                tabela.row([str(item) for item in data_row])

        # --- ETAPA 4: Salva o Arquivo ---
        pdf.output(caminho_arquivo)
        print(f"✔ Relatório em PDF salvo com sucesso!")

    except Exception as e:
        print(f"❌ Erro ao salvar o PDF: {e}")

In [17]:
ARQUIVO_SAIDA_PDF = r"C:\Dev\Testes\Relatorio_Validacao_Enderecos.pdf"
salvar_df_como_pdf(df_resultado_final, ARQUIVO_SAIDA_PDF)

Iniciando a geração do PDF profissional em: C:\Dev\Testes\Relatorio_Validacao_Enderecos.pdf...
✔ Relatório em PDF salvo com sucesso!
